In [ ]:
# In[1]:


import pandas as pd
from datetime import date
import numpy as np
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
#pd.set_option('display.float_format',lambda x: "{:,.2f}".format(x))


# In[3]:


#Input formato
formato=input('Ingresa formato en minusculas, valores permitidos (ba, bae, mb, sc, sp-wme): ')


# In[355]:


#Leer tablas base en csv y demás tablas o catalogos utilizados
tera=pd.read_csv('tablas_base/%s_altertab.csv'%formato,dtype={'DEPTO':str,'CATEGORIA':str})
tram=pd.read_csv('tablas_base/tramos_altertab.csv',dtype={'DEPTO':str,'CATEGORIA':str})


# In[356]:


### Input determinante
det=input('Ingresa det: ')


# In[357]:


cat_lim=pd.read_excel('tablas_base/Catalogos Altertab.xlsx',sheet_name='LIMITES',dtype={'DEPTO':str,'CATEGORIA':str})
muebles=pd.read_csv('tablas_base/cat_muebles.csv',dtype={'DEPTO':str,'CATEGORIA':str})
directorio=pd.read_excel('tablas_base/Directorio de tiendas.xlsx',sheet_name='LISTADO DE TIENDAS',usecols=['DETERMINANTE','ESTADO','ZONANIELSEN','PROTO']).rename(columns={'DETERMINANTE':'DET'})
tramos_tienda=pd.read_excel('tablas_base/tt_tienda.xlsx',sheet_name='desempeno',dtype={'DEPTO':str,'CATEGORIA':str})
fc=pd.read_excel('tablas_base/Catalogos Altertab.xlsx',sheet_name='FC',dtype={'TRAMOS':float})


# In[358]:


#Se convierte input a entero y se captura la fecha actual en una variable
detint=int(det)
fecha=date.today().strftime("%d-%m-%Y")


# In[359]:


#Filtro
tera=tera.loc[tera['DET']==detint]
tram=tram.loc[tram['DET']==detint]


# In[360]:


#Redondeo a 4 decimales para las columnas del margen
tera['NSQ']=tera['NSQ'].round(4)
tera['NSC']=tera['NSC'].round(4)
tera['NSR']=tera['NSR'].round(4)


# In[361]:


#Vistazo de las tablas filtradas
print(tera.shape)
tera.head(2)


# In[362]:


#Con esta funcion lambda se cambia el nombre de  los registros donde el mueble es capacity cero, replicando asi el manejo de 
#muebles capacity cero del query de teradata altertab 2.0
tera['TIPO_MUEBLE']= tera.apply(lambda x: x['TIPO_MUEBLE']+'_cap0_d'+x['DEPTO']+'c'+x['CATEGORIA'] if x['CAPACIDAD']==0 else x['TIPO_MUEBLE'] ,axis=1)


# In[363]:


#Se genera tabla separada con muebles capacity cero
mblcap0=tera[tera['CAPACIDAD']==0].copy()
mblcap0=mblcap0[['DET','FLOORPLAN','DEPTO','CATEGORIA','NOMBRE_CATEGORIA','TIPO_MUEBLE','TRAMOS_CONV']]
#mblcap0


# In[364]:


#Se guitan los capacity cero del analisis
tera=tera[tera['CAPACIDAD']>0].copy()


# In[365]:


#El margen se calcula con las formulas oficiales, esta funcion ajusta un poco mejor a aquellos articulos cuyo margen inicial
#es menor a cero
def calcula_margen(t):
    if t['TOTAL_MUMD'] > 0:
        return t['MARGEN_INICIAL']-((t['TOTAL_MUMD']/t['VTA_RETAIL'])*(1-t['MARGEN_INICIAL']))
    else:
        return t['MARGEN_INICIAL']-((t['TOTAL_MUMD']/t['VTA_RETAIL'])*(1+t['MARGEN_INICIAL']))

tera['NSR'][tera['NSR']==0]=np.nan
tera['VTA_RETAIL'][tera['VTA_RETAIL']==0]=np.nan
tera['MARGEN_INICIAL']=1-(tera['NSC']/tera['NSR'])
tera['MARGEN_MANTENIDO']=tera.apply(calcula_margen,axis=1)
tera['MARGEN_MANTENIDO'][tera['MARGEN_MANTENIDO']<0]=0


# In[366]:


#Esto es lo mismo que OVER(PARTITION BY) en teradata, es un paso fundamental ya que es aqui donde se agrupa la tabla
#al nivel requerido para que juegue solo ciertas columnas (muebles,floorplans, etc) y no se mezclen entre si
overp1=tera[['FLOORPLAN','TIPO_MUEBLE','TRAMOS_CONV','VTA_PIEZAS','VTA_RETAIL','VTA_VOLUMEN','MARGEN_MANTENIDO']].groupby(['FLOORPLAN','TIPO_MUEBLE']).agg('sum').reset_index()
tera=tera.merge(overp1, on=['FLOORPLAN','TIPO_MUEBLE'],suffixes=('','_TOT'))


# In[367]:


print(tera.shape)
tera.head(2)


# In[368]:


#Vistazo de ventas por departamento y categoria
res_dep=tera[['FLOORPLAN','TRAMOS_CONV','VTA_RETAIL','VTA_PIEZAS','VTA_VOLUMEN','MARGEN_MANTENIDO','TENDENCIA']].groupby(['FLOORPLAN']).agg({'TRAMOS_CONV':np.sum,'VTA_RETAIL':np.sum,'VTA_PIEZAS':np.sum,'VTA_VOLUMEN':np.sum,'TENDENCIA':np.mean,'MARGEN_MANTENIDO':np.mean})
res_dep['PCT_RETAIL']=res_dep['VTA_RETAIL']/res_dep['VTA_RETAIL'].sum()*100.0
res_dep['PCT_PIEZAS']=res_dep['VTA_PIEZAS']/res_dep['VTA_PIEZAS'].sum()*100.0
res_dep['PCT_VOLUMEN']=res_dep['VTA_VOLUMEN']/res_dep['VTA_VOLUMEN'].sum()*100.0
res_dep['PCT_MARGEN']=res_dep['MARGEN_MANTENIDO']/res_dep['MARGEN_MANTENIDO'].sum()*100.0
res_dep['ASIGNACION']=(res_dep['PCT_RETAIL']*0.3)+(res_dep['PCT_PIEZAS']*0.2)+(res_dep['PCT_VOLUMEN']*0.2)+(res_dep['PCT_MARGEN']*0.3)
res_dep['PCT_TRAM']=res_dep['TRAMOS_CONV']/res_dep['TRAMOS_CONV'].sum()*100.0
res_dep['DIFF']=res_dep['ASIGNACION']-res_dep['PCT_TRAM']


# In[369]:


#DIFF negativo es mas venta que tramos, positivo es mas tramos que venta
print(abs(res_dep['DIFF']).sum())
res_dep=res_dep.sort_values('DIFF',ascending=False).round(2)
res_dep


# In[370]:


cat_lim.drop('NOMBRE_CATEGORIA',axis=1,inplace=True)
tera=tera.merge(cat_lim,on=['FORMATO','DEPTO','CATEGORIA'],how='left')


# In[371]:


def asiglim(t):
    if t['ASIGNACION'] < t['TRAMOS_MIN']:
        return t['TRAMOS_MIN']
    elif t['ASIGNACION'] > t['TRAMOS_MAX']:
        return t['TRAMOS_MAX']
    else:
        return t['ASIGNACION']


# In[372]:


#Calculos para el analisis
tera['SH_TRAMOS']=tera['TRAMOS_CONV']/tera['TRAMOS_CONV_TOT']
tera['SH_PIEZAS']=tera['VTA_PIEZAS']/tera['VTA_PIEZAS_TOT']
tera['SH_RETAIL']=tera['VTA_RETAIL']/tera['VTA_RETAIL_TOT']
tera['SH_VOLUMEN']=tera['VTA_VOLUMEN']/tera['VTA_VOLUMEN_TOT']
tera['SH_MARGEN']=tera['MARGEN_MANTENIDO']/tera['MARGEN_MANTENIDO_TOT']

tera['ASIGNACION_PIEZAS']=tera['TRAMOS_CONV_TOT']*tera['SH_PIEZAS']
tera['ASIGNACION_RETAIL']=tera['TRAMOS_CONV_TOT']*tera['SH_RETAIL']
tera['ASIGNACION_VOLUMEN']=tera['TRAMOS_CONV_TOT']*tera['SH_VOLUMEN']
tera['ASIGNACION_MARGEN']=tera['TRAMOS_CONV_TOT']*tera['SH_MARGEN']

#tera['DESBALANCE_PIEZAS']=abs(tera['SH_TRAMOS']-tera['SH_PIEZAS'])
#tera['DESBALANCE_RETAIL']=abs(tera['SH_TRAMOS']-tera['SH_RETAIL'])

tera['VTAXTRAMO_PIEZAS']=tera['VTA_PIEZAS']/tera['TRAMOS_CONV']
tera['VTAXTRAMO_RETAIL']=tera['VTA_RETAIL']/tera['TRAMOS_CONV']

tera['DIM']=(tera['CAPACIDAD']/tera['VTA_PIEZAS'])*7
#Sustituir dims infinitos y dims menores a cero con nulos
tera['DIM'].replace(float('inf'),np.nan,inplace=True)
tera['DIM'][tera['DIM']<0]=np.nan
tera['DIMXTRAMO']=tera['DIM']/tera['TRAMOS_CONV']
#tera['INVENTORY_TURNOVER']=30/tera['DIM']

tera['ASIGNACION']=(tera['ASIGNACION_RETAIL']*0.30)+(tera['ASIGNACION_PIEZAS']*0.20)+(tera['ASIGNACION_VOLUMEN']*0.20)+(tera['ASIGNACION_MARGEN']*0.30)
tera['ASIG_ORIGINAL']=tera['ASIGNACION'].copy()
tera['ASIGNACION+LIMITE']=tera.apply(asiglim,axis=1)


# In[373]:


tera['DIFFS']=tera['ASIGNACION']-tera['ASIGNACION+LIMITE']
#aux2=fijos, aux1=pendientes
aux2=tera.loc[~(tera['DIFFS']==0)].copy()
aux2['ASIGNACION']=aux2['ASIGNACION+LIMITE'].copy()
aux1=tera.loc[tera['DIFFS']==0].copy()
overp2=aux1[['FLOORPLAN','TIPO_MUEBLE','TRAMOS_CONV','MARGEN_MANTENIDO']].groupby(['FLOORPLAN','TIPO_MUEBLE']).agg('sum').reset_index()
overp2['SH_TRAMOS']=overp2['TRAMOS_CONV']/overp2['TRAMOS_CONV'].sum()
overp2['TRAMOS_POR_ASIGNAR']=tera['TRAMOS_CONV'].sum()-aux2['ASIGNACION+LIMITE'].sum()
overp2['TRAMOS_AUX']=overp2['TRAMOS_POR_ASIGNAR']*overp2['SH_TRAMOS']
overp2.drop(columns=['SH_TRAMOS','TRAMOS_POR_ASIGNAR','TRAMOS_CONV'],inplace=True)
aux1.drop(columns=[x for x in aux1.columns if x[-3:]=='TOT'],inplace=True)
aux1=aux1.merge(overp2, on=['FLOORPLAN','TIPO_MUEBLE'],suffixes=('','_TOT'),how='left')
aux1['SH_MARGEN']=aux1['MARGEN_MANTENIDO']/aux1['MARGEN_MANTENIDO_TOT']
aux1['ASIGNACION_AUX']=aux1['SH_MARGEN']*aux1['TRAMOS_AUX']
aux1['ASIGNACION']=aux1['ASIGNACION_AUX'].copy()
aux1['ASIGNACION+LIMITE']=aux1.apply(asiglim,axis=1)


# In[374]:


for x in range(3):
    aux1['DIFFS']=aux1['ASIGNACION']-aux1['ASIGNACION+LIMITE']
    aux3=aux1.loc[~(aux1['DIFFS']==0)].copy()
    aux3['ASIGNACION']=aux3.apply(asiglim,axis=1)
    aux2=pd.concat([aux2,aux3])
    aux1=aux1.loc[aux1['DIFFS']==0].copy()
    overp2=aux1[['FLOORPLAN','TIPO_MUEBLE','TRAMOS_CONV','MARGEN_MANTENIDO']].groupby(['FLOORPLAN','TIPO_MUEBLE']).agg('sum').reset_index()
    overp2['SH_TRAMOS']=overp2['TRAMOS_CONV']/overp2['TRAMOS_CONV'].sum()
    overp2['TRAMOS_POR_ASIGNAR']=tera['TRAMOS_CONV'].sum()-aux2['ASIGNACION+LIMITE'].sum()
    overp2['TRAMOS_AUX']=overp2['TRAMOS_POR_ASIGNAR']*overp2['SH_TRAMOS']
    overp2.drop(columns=['SH_TRAMOS','TRAMOS_POR_ASIGNAR','TRAMOS_CONV'],inplace=True)
    aux1.drop(columns=[x for x in aux1.columns if x[-3:]=='TOT'],inplace=True)
    aux1=aux1.merge(overp2, on=['FLOORPLAN','TIPO_MUEBLE'],suffixes=('','_TOT'),how='left')
    aux1['SH_MARGEN']=aux1['MARGEN_MANTENIDO']/aux1['MARGEN_MANTENIDO_TOT']
    aux1['ASIGNACION_AUX']=aux1['SH_MARGEN']*aux1['TRAMOS_AUX_TOT']
    aux1['ASIGNACION']=aux1['ASIGNACION_AUX'].copy()
    aux1['ASIGNACION+LIMITE']=aux1.apply(asiglim,axis=1)


# In[375]:


g1=pd.concat([aux1,aux2])
g1['ASIGNACION']=g1['ASIG_ORIGINAL'].copy()
g1['VARIACION_TRAMOS']=g1['ASIGNACION']-g1['TRAMOS_CONV']
g1['PCT_VARIACION_TRAMOS']=g1['VARIACION_TRAMOS']/g1['TRAMOS_CONV']
g1['VARIACION_DIMS']=(g1['PCT_VARIACION_TRAMOS']+1)*g1['DIM']


# In[376]:


#Se agregan los tramos originales, nombre original del mueble y estado
output=g1.merge(tram[['FLOORPLAN','DEPTO','CATEGORIA','TIPO_MUEBLE','TRAMOS']],on=['FLOORPLAN','DEPTO','CATEGORIA','TIPO_MUEBLE'],how='left')
output=output.merge(muebles,on=['DET','DEPTO','CATEGORIA'],how='left')
output=output.merge(directorio,on='DET',how='left')
output['FC']=output['TRAMOS_CONV']/output['TRAMOS']
output['ASIGNACION_MUEBLE']=output['ASIGNACION']/output['FC']
output['VARIACION_2']=output['ASIGNACION_MUEBLE']-output['TRAMOS']
output['PCT_VARIACION_2']=output['VARIACION_2']/output['TRAMOS']


# In[377]:


#Desbalance piezas y retail
#print(output['DESBALANCE_PIEZAS'].sum())
#print(output['DESBALANCE_RETAIL'].sum())


# In[378]:


#Generacion de la tabla para indicadores de desempeño en ckb
#Esta tabla pivote es para obtener combinaciones depto-categoria unicas del analisis
output_ckb=output.pivot_table(index=['DEPTO','CATEGORIA'],values=['DIMXTRAMO','VTAXTRAMO_PIEZAS','VTAXTRAMO_RETAIL']
                ,aggfunc={'DIMXTRAMO':'sum','VTAXTRAMO_PIEZAS':'sum','VTAXTRAMO_RETAIL':'sum'}).reset_index()
#La tabla que se lee del excel manual se cruza con la tabla pivote anterios
tabla_ckb=tramos_tienda[['DEPTO','CATEGORIA','NOMBRE_CATEGORIA','TRAMOS','NOMBRE_MUEBLE']].merge(output_ckb
                ,on=['DEPTO','CATEGORIA'],how='left')
#Se agrega el factor de conversion
tabla_ckb=tabla_ckb.merge(fc,on='NOMBRE_MUEBLE',how='left')
tabla_ckb['FACTOR_CONVERSION'].fillna(1,inplace=True)
tabla_ckb=tabla_ckb[['DEPTO','CATEGORIA','NOMBRE_CATEGORIA','TRAMOS', 'NOMBRE_MUEBLE','FACTOR_CONVERSION','DIMXTRAMO'
                     ,'VTAXTRAMO_PIEZAS', 'VTAXTRAMO_RETAIL']]


# In[379]:


#Se agrega el simbolo % a las columnas de porcentaje y se redondea a 2 decimales
output[['SH_TRAMOS','SH_PIEZAS','SH_RETAIL','SH_VOLUMEN','SH_MARGEN','PCT_VARIACION_TRAMOS']]=(output[['SH_TRAMOS','SH_PIEZAS'
        ,'SH_RETAIL','SH_VOLUMEN','SH_MARGEN','PCT_VARIACION_TRAMOS']]*100).round(2).astype(str)+'%'


# In[380]:


#Reacomodo de columnas y redondeo a 2 decimales
output=output[['ZONANIELSEN','ESTADO','PROTO','DET','STORE_NAME','FLOORPLAN','DEPTO','CATEGORIA','NOMBRE_CATEGORIA','TIPO_MUEBLE','TRAMOS'
        ,'TRAMOS_CONV','CAPACIDAD','VTA_PIEZAS','VTA_RETAIL','VTA_VOLUMEN','MARGEN_MANTENIDO','SH_TRAMOS'
        ,'SH_PIEZAS','SH_RETAIL','SH_VOLUMEN','SH_MARGEN','CONTEO_UPC','VTAXTRAMO_PIEZAS','VTAXTRAMO_RETAIL','TENDENCIA','DIM'
        ,'DIMXTRAMO','TRAMOS_MIN','TRAMOS_MAX','ASIGNACION_PIEZAS','ASIGNACION_RETAIL','ASIGNACION_VOLUMEN','ASIGNACION_MARGEN'
        ,'ASIGNACION','ASIGNACION+LIMITE','VARIACION_DIMS','VARIACION_TRAMOS','PCT_VARIACION_TRAMOS','NOMBRE_MUEBLE','ASIGNACION_MUEBLE'
        ,'VARIACION_2','PCT_VARIACION_2']].round(2)


# In[381]:


#Exportar a excel
with pd.ExcelWriter(f'altertab_{detint}({fecha}).xlsx') as writer:
    output.sort_values('FLOORPLAN',ascending=True).to_excel(writer, sheet_name='BASE',index=False)
    mblcap0.to_excel(writer, sheet_name='MBLCAP_0',index=False)
    res_dep.to_excel(writer, sheet_name='RESUMEN_DEPTO')
    #tabla_ckb.to_excel(writer, sheet_name='TABLA_DESEMPENO',index=False)


# coding: utf-8

# In[1]:


import pandas as pd
from datetime import date
import numpy as np
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
#pd.set_option('display.float_format',lambda x: "{:,.2f}".format(x))


# In[2]:


print(' _____ _                        _\n|  ___| |                      | |\n| |_  | | ___   ___  _ __ _ __ | | __ _ _ __  ___\n|  _| | |/ _ \ / _ \| \'__| \'_ \| |/ _` | \'_ \/ __|\n| |   | | (_) | (_) | |  | |_) | | (_| | | | \__ \ \n\_|   |_|\___/ \___/|_|  | .__/|_|\__,_|_| |_|___/\n                         | |\n                         |_|\n')
print('<<Altertab ventas por determinante y floorplan>>  v3.6')


# In[3]:


#Input formato
formato=input('Ingresa formato en minusculas, valores permitidos (ba, bae, mb, sc, sp-wme): ')


# In[355]:


#Leer tablas base en csv y demás tablas o catalogos utilizados
tera=pd.read_csv('tablas_base/%s_altertab.csv'%formato,dtype={'DEPTO':str,'CATEGORIA':str})
tram=pd.read_csv('tablas_base/tramos_altertab.csv',dtype={'DEPTO':str,'CATEGORIA':str})


# In[356]:


### Input determinante
det=input('Ingresa det: ')


# In[357]:


cat_lim=pd.read_excel('tablas_base/Catalogos Altertab.xlsx',sheet_name='LIMITES',dtype={'DEPTO':str,'CATEGORIA':str})
muebles=pd.read_csv('tablas_base/cat_muebles.csv',dtype={'DEPTO':str,'CATEGORIA':str})
directorio=pd.read_excel('tablas_base/Directorio de tiendas.xlsx',sheet_name='LISTADO DE TIENDAS',usecols=['DETERMINANTE','ESTADO','ZONANIELSEN','PROTO']).rename(columns={'DETERMINANTE':'DET'})
tramos_tienda=pd.read_excel('tablas_base/tt_tienda.xlsx',sheet_name='desempeno',dtype={'DEPTO':str,'CATEGORIA':str})
fc=pd.read_excel('tablas_base/Catalogos Altertab.xlsx',sheet_name='FC',dtype={'TRAMOS':float})


# In[358]:


#Se convierte input a entero y se captura la fecha actual en una variable
detint=int(det)
fecha=date.today().strftime("%d-%m-%Y")


# In[359]:


#Filtro
tera=tera.loc[tera['DET']==detint]
tram=tram.loc[tram['DET']==detint]


# In[360]:


#Redondeo a 4 decimales para las columnas del margen
tera['NSQ']=tera['NSQ'].round(4)
tera['NSC']=tera['NSC'].round(4)
tera['NSR']=tera['NSR'].round(4)


# In[361]:


#Vistazo de las tablas filtradas
print(tera.shape)
tera.head(2)


# In[362]:


#Con esta funcion lambda se cambia el nombre de  los registros donde el mueble es capacity cero, replicando asi el manejo de 
#muebles capacity cero del query de teradata altertab 2.0
tera['TIPO_MUEBLE']= tera.apply(lambda x: x['TIPO_MUEBLE']+'_cap0_d'+x['DEPTO']+'c'+x['CATEGORIA'] if x['CAPACIDAD']==0 else x['TIPO_MUEBLE'] ,axis=1)


# In[363]:


#Se genera tabla separada con muebles capacity cero
mblcap0=tera[tera['CAPACIDAD']==0].copy()
mblcap0=mblcap0[['DET','FLOORPLAN','DEPTO','CATEGORIA','NOMBRE_CATEGORIA','TIPO_MUEBLE','TRAMOS_CONV']]
#mblcap0


# In[364]:


#Se guitan los capacity cero del analisis
tera=tera[tera['CAPACIDAD']>0].copy()


# In[365]:


#El margen se calcula con las formulas oficiales, esta funcion ajusta un poco mejor a aquellos articulos cuyo margen inicial
#es menor a cero
def calcula_margen(t):
    if t['TOTAL_MUMD'] > 0:
        return t['MARGEN_INICIAL']-((t['TOTAL_MUMD']/t['VTA_RETAIL'])*(1-t['MARGEN_INICIAL']))
    else:
        return t['MARGEN_INICIAL']-((t['TOTAL_MUMD']/t['VTA_RETAIL'])*(1+t['MARGEN_INICIAL']))

tera['NSR'][tera['NSR']==0]=np.nan
tera['VTA_RETAIL'][tera['VTA_RETAIL']==0]=np.nan
tera['MARGEN_INICIAL']=1-(tera['NSC']/tera['NSR'])
tera['MARGEN_MANTENIDO']=tera.apply(calcula_margen,axis=1)
tera['MARGEN_MANTENIDO'][tera['MARGEN_MANTENIDO']<0]=0


# In[366]:


#Esto es lo mismo que OVER(PARTITION BY) en teradata, es un paso fundamental ya que es aqui donde se agrupa la tabla
#al nivel requerido para que juegue solo ciertas columnas (muebles,floorplans, etc) y no se mezclen entre si
overp1=tera[['FLOORPLAN','TIPO_MUEBLE','TRAMOS_CONV','VTA_PIEZAS','VTA_RETAIL','VTA_VOLUMEN','MARGEN_MANTENIDO']].groupby(['FLOORPLAN','TIPO_MUEBLE']).agg('sum').reset_index()
tera=tera.merge(overp1, on=['FLOORPLAN','TIPO_MUEBLE'],suffixes=('','_TOT'))


# In[367]:


print(tera.shape)
tera.head(2)


# In[368]:


#Vistazo de ventas por departamento y categoria
res_dep=tera[['FLOORPLAN','TRAMOS_CONV','VTA_RETAIL','VTA_PIEZAS','VTA_VOLUMEN','MARGEN_MANTENIDO','TENDENCIA']].groupby(['FLOORPLAN']).agg({'TRAMOS_CONV':np.sum,'VTA_RETAIL':np.sum,'VTA_PIEZAS':np.sum,'VTA_VOLUMEN':np.sum,'TENDENCIA':np.mean,'MARGEN_MANTENIDO':np.mean})
res_dep['PCT_RETAIL']=res_dep['VTA_RETAIL']/res_dep['VTA_RETAIL'].sum()*100.0
res_dep['PCT_PIEZAS']=res_dep['VTA_PIEZAS']/res_dep['VTA_PIEZAS'].sum()*100.0
res_dep['PCT_VOLUMEN']=res_dep['VTA_VOLUMEN']/res_dep['VTA_VOLUMEN'].sum()*100.0
res_dep['PCT_MARGEN']=res_dep['MARGEN_MANTENIDO']/res_dep['MARGEN_MANTENIDO'].sum()*100.0
res_dep['ASIGNACION']=(res_dep['PCT_RETAIL']*0.3)+(res_dep['PCT_PIEZAS']*0.2)+(res_dep['PCT_VOLUMEN']*0.2)+(res_dep['PCT_MARGEN']*0.3)
res_dep['PCT_TRAM']=res_dep['TRAMOS_CONV']/res_dep['TRAMOS_CONV'].sum()*100.0
res_dep['DIFF']=res_dep['ASIGNACION']-res_dep['PCT_TRAM']


# In[369]:


#DIFF negativo es mas venta que tramos, positivo es mas tramos que venta
print(abs(res_dep['DIFF']).sum())
res_dep=res_dep.sort_values('DIFF',ascending=False).round(2)
res_dep


# In[370]:


cat_lim.drop('NOMBRE_CATEGORIA',axis=1,inplace=True)
tera=tera.merge(cat_lim,on=['FORMATO','DEPTO','CATEGORIA'],how='left')


# In[371]:


def asiglim(t):
    if t['ASIGNACION'] < t['TRAMOS_MIN']:
        return t['TRAMOS_MIN']
    elif t['ASIGNACION'] > t['TRAMOS_MAX']:
        return t['TRAMOS_MAX']
    else:
        return t['ASIGNACION']


# In[372]:


#Calculos para el analisis
tera['SH_TRAMOS']=tera['TRAMOS_CONV']/tera['TRAMOS_CONV_TOT']
tera['SH_PIEZAS']=tera['VTA_PIEZAS']/tera['VTA_PIEZAS_TOT']
tera['SH_RETAIL']=tera['VTA_RETAIL']/tera['VTA_RETAIL_TOT']
tera['SH_VOLUMEN']=tera['VTA_VOLUMEN']/tera['VTA_VOLUMEN_TOT']
tera['SH_MARGEN']=tera['MARGEN_MANTENIDO']/tera['MARGEN_MANTENIDO_TOT']

tera['ASIGNACION_PIEZAS']=tera['TRAMOS_CONV_TOT']*tera['SH_PIEZAS']
tera['ASIGNACION_RETAIL']=tera['TRAMOS_CONV_TOT']*tera['SH_RETAIL']
tera['ASIGNACION_VOLUMEN']=tera['TRAMOS_CONV_TOT']*tera['SH_VOLUMEN']
tera['ASIGNACION_MARGEN']=tera['TRAMOS_CONV_TOT']*tera['SH_MARGEN']

#tera['DESBALANCE_PIEZAS']=abs(tera['SH_TRAMOS']-tera['SH_PIEZAS'])
#tera['DESBALANCE_RETAIL']=abs(tera['SH_TRAMOS']-tera['SH_RETAIL'])

tera['VTAXTRAMO_PIEZAS']=tera['VTA_PIEZAS']/tera['TRAMOS_CONV']
tera['VTAXTRAMO_RETAIL']=tera['VTA_RETAIL']/tera['TRAMOS_CONV']

tera['DIM']=(tera['CAPACIDAD']/tera['VTA_PIEZAS'])*7
#Sustituir dims infinitos y dims menores a cero con nulos
tera['DIM'].replace(float('inf'),np.nan,inplace=True)
tera['DIM'][tera['DIM']<0]=np.nan
tera['DIMXTRAMO']=tera['DIM']/tera['TRAMOS_CONV']
#tera['INVENTORY_TURNOVER']=30/tera['DIM']

tera['ASIGNACION']=(tera['ASIGNACION_RETAIL']*0.30)+(tera['ASIGNACION_PIEZAS']*0.20)+(tera['ASIGNACION_VOLUMEN']*0.20)+(tera['ASIGNACION_MARGEN']*0.30)
tera['ASIG_ORIGINAL']=tera['ASIGNACION'].copy()
tera['ASIGNACION+LIMITE']=tera.apply(asiglim,axis=1)


# In[373]:


tera['DIFFS']=tera['ASIGNACION']-tera['ASIGNACION+LIMITE']
#aux2=fijos, aux1=pendientes
aux2=tera.loc[~(tera['DIFFS']==0)].copy()
aux2['ASIGNACION']=aux2['ASIGNACION+LIMITE'].copy()
aux1=tera.loc[tera['DIFFS']==0].copy()
overp2=aux1[['FLOORPLAN','TIPO_MUEBLE','TRAMOS_CONV','MARGEN_MANTENIDO']].groupby(['FLOORPLAN','TIPO_MUEBLE']).agg('sum').reset_index()
overp2['SH_TRAMOS']=overp2['TRAMOS_CONV']/overp2['TRAMOS_CONV'].sum()
overp2['TRAMOS_POR_ASIGNAR']=tera['TRAMOS_CONV'].sum()-aux2['ASIGNACION+LIMITE'].sum()
overp2['TRAMOS_AUX']=overp2['TRAMOS_POR_ASIGNAR']*overp2['SH_TRAMOS']
overp2.drop(columns=['SH_TRAMOS','TRAMOS_POR_ASIGNAR','TRAMOS_CONV'],inplace=True)
aux1.drop(columns=[x for x in aux1.columns if x[-3:]=='TOT'],inplace=True)
aux1=aux1.merge(overp2, on=['FLOORPLAN','TIPO_MUEBLE'],suffixes=('','_TOT'),how='left')
aux1['SH_MARGEN']=aux1['MARGEN_MANTENIDO']/aux1['MARGEN_MANTENIDO_TOT']
aux1['ASIGNACION_AUX']=aux1['SH_MARGEN']*aux1['TRAMOS_AUX']
aux1['ASIGNACION']=aux1['ASIGNACION_AUX'].copy()
aux1['ASIGNACION+LIMITE']=aux1.apply(asiglim,axis=1)


# In[374]:


for x in range(3):
    aux1['DIFFS']=aux1['ASIGNACION']-aux1['ASIGNACION+LIMITE']
    aux3=aux1.loc[~(aux1['DIFFS']==0)].copy()
    aux3['ASIGNACION']=aux3.apply(asiglim,axis=1)
    aux2=pd.concat([aux2,aux3])
    aux1=aux1.loc[aux1['DIFFS']==0].copy()
    overp2=aux1[['FLOORPLAN','TIPO_MUEBLE','TRAMOS_CONV','MARGEN_MANTENIDO']].groupby(['FLOORPLAN','TIPO_MUEBLE']).agg('sum').reset_index()
    overp2['SH_TRAMOS']=overp2['TRAMOS_CONV']/overp2['TRAMOS_CONV'].sum()
    overp2['TRAMOS_POR_ASIGNAR']=tera['TRAMOS_CONV'].sum()-aux2['ASIGNACION+LIMITE'].sum()
    overp2['TRAMOS_AUX']=overp2['TRAMOS_POR_ASIGNAR']*overp2['SH_TRAMOS']
    overp2.drop(columns=['SH_TRAMOS','TRAMOS_POR_ASIGNAR','TRAMOS_CONV'],inplace=True)
    aux1.drop(columns=[x for x in aux1.columns if x[-3:]=='TOT'],inplace=True)
    aux1=aux1.merge(overp2, on=['FLOORPLAN','TIPO_MUEBLE'],suffixes=('','_TOT'),how='left')
    aux1['SH_MARGEN']=aux1['MARGEN_MANTENIDO']/aux1['MARGEN_MANTENIDO_TOT']
    aux1['ASIGNACION_AUX']=aux1['SH_MARGEN']*aux1['TRAMOS_AUX_TOT']
    aux1['ASIGNACION']=aux1['ASIGNACION_AUX'].copy()
    aux1['ASIGNACION+LIMITE']=aux1.apply(asiglim,axis=1)


# In[375]:


g1=pd.concat([aux1,aux2])
g1['ASIGNACION']=g1['ASIG_ORIGINAL'].copy()
g1['VARIACION_TRAMOS']=g1['ASIGNACION']-g1['TRAMOS_CONV']
g1['PCT_VARIACION_TRAMOS']=g1['VARIACION_TRAMOS']/g1['TRAMOS_CONV']
g1['VARIACION_DIMS']=(g1['PCT_VARIACION_TRAMOS']+1)*g1['DIM']


# In[376]:


#Se agregan los tramos originales, nombre original del mueble y estado
output=g1.merge(tram[['FLOORPLAN','DEPTO','CATEGORIA','TIPO_MUEBLE','TRAMOS']],on=['FLOORPLAN','DEPTO','CATEGORIA','TIPO_MUEBLE'],how='left')
output=output.merge(muebles,on=['DET','DEPTO','CATEGORIA'],how='left')
output=output.merge(directorio,on='DET',how='left')
output['FC']=output['TRAMOS_CONV']/output['TRAMOS']
output['ASIGNACION_MUEBLE']=output['ASIGNACION']/output['FC']
output['VARIACION_2']=output['ASIGNACION_MUEBLE']-output['TRAMOS']
output['PCT_VARIACION_2']=output['VARIACION_2']/output['TRAMOS']


# In[377]:


#Desbalance piezas y retail
#print(output['DESBALANCE_PIEZAS'].sum())
#print(output['DESBALANCE_RETAIL'].sum())


# In[378]:


#Generacion de la tabla para indicadores de desempeño en ckb
#Esta tabla pivote es para obtener combinaciones depto-categoria unicas del analisis
output_ckb=output.pivot_table(index=['DEPTO','CATEGORIA'],values=['DIMXTRAMO','VTAXTRAMO_PIEZAS','VTAXTRAMO_RETAIL']
                ,aggfunc={'DIMXTRAMO':'sum','VTAXTRAMO_PIEZAS':'sum','VTAXTRAMO_RETAIL':'sum'}).reset_index()
#La tabla que se lee del excel manual se cruza con la tabla pivote anterios
tabla_ckb=tramos_tienda[['DEPTO','CATEGORIA','NOMBRE_CATEGORIA','TRAMOS','NOMBRE_MUEBLE']].merge(output_ckb
                ,on=['DEPTO','CATEGORIA'],how='left')
#Se agrega el factor de conversion
tabla_ckb=tabla_ckb.merge(fc,on='NOMBRE_MUEBLE',how='left')
tabla_ckb['FACTOR_CONVERSION'].fillna(1,inplace=True)
tabla_ckb=tabla_ckb[['DEPTO','CATEGORIA','NOMBRE_CATEGORIA','TRAMOS', 'NOMBRE_MUEBLE','FACTOR_CONVERSION','DIMXTRAMO'
                     ,'VTAXTRAMO_PIEZAS', 'VTAXTRAMO_RETAIL']]


# In[379]:


#Se agrega el simbolo % a las columnas de porcentaje y se redondea a 2 decimales
output[['SH_TRAMOS','SH_PIEZAS','SH_RETAIL','SH_VOLUMEN','SH_MARGEN','PCT_VARIACION_TRAMOS']]=(output[['SH_TRAMOS','SH_PIEZAS'
        ,'SH_RETAIL','SH_VOLUMEN','SH_MARGEN','PCT_VARIACION_TRAMOS']]*100).round(2).astype(str)+'%'


# In[380]:


#Reacomodo de columnas y redondeo a 2 decimales
output=output[['ZONANIELSEN','ESTADO','PROTO','DET','STORE_NAME','FLOORPLAN','DEPTO','CATEGORIA','NOMBRE_CATEGORIA','TIPO_MUEBLE','TRAMOS'
        ,'TRAMOS_CONV','CAPACIDAD','VTA_PIEZAS','VTA_RETAIL','VTA_VOLUMEN','MARGEN_MANTENIDO','SH_TRAMOS'
        ,'SH_PIEZAS','SH_RETAIL','SH_VOLUMEN','SH_MARGEN','CONTEO_UPC','VTAXTRAMO_PIEZAS','VTAXTRAMO_RETAIL','TENDENCIA','DIM'
        ,'DIMXTRAMO','TRAMOS_MIN','TRAMOS_MAX','ASIGNACION_PIEZAS','ASIGNACION_RETAIL','ASIGNACION_VOLUMEN','ASIGNACION_MARGEN'
        ,'ASIGNACION','ASIGNACION+LIMITE','VARIACION_DIMS','VARIACION_TRAMOS','PCT_VARIACION_TRAMOS','NOMBRE_MUEBLE','ASIGNACION_MUEBLE'
        ,'VARIACION_2','PCT_VARIACION_2']].round(2)


# In[381]:


#Exportar a excel
with pd.ExcelWriter(f'altertab_{detint}({fecha}).xlsx') as writer:
    output.sort_values('FLOORPLAN',ascending=True).to_excel(writer, sheet_name='BASE',index=False)
    mblcap0.to_excel(writer, sheet_name='MBLCAP_0',index=False)
    res_dep.to_excel(writer, sheet_name='RESUMEN_DEPTO')
    #tabla_ckb.to_excel(writer, sheet_name='TABLA_DESEMPENO',index=False)